In [80]:
# Importing basic libraries for data manipulation and analysis
import numpy as np                # For numerical operations
import pandas as pd # For data manipulation and analysis

# Importing libraries for data visualization
import matplotlib.pyplot as plt   # For basic plotting
import seaborn as sns             # For advanced and aesthetically pleasing visualizations

# Importing libraries for statistical analysis
import scipy.stats as stats       # For statistical tests and distributions
from statsmodels.tsa.stattools import adfuller  # For time series analysis (stationarity test)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # For autocorrelation and partial autocorrelation plots

# Importing libraries for machine learning (optional, for advanced EDA)
from sklearn.preprocessing import LabelEncoder, StandardScaler  # For preprocessing
# Import label encoder 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split  # For splitting data into train/test sets
from sklearn.decomposition import PCA  # For dimensionality reduction

# Setting up display options
pd.set_option('display.max_columns', None)  # Display all columns in DataFrame
pd.set_option('display.max_rows', 100)  # Set the number of rows to display
sns.set(style="whitegrid")  # Set Seaborn style for plots

# Warnings
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings to keep the output clean


In [81]:
customer_info_df = pd.read_csv("/Users/Jamie/OneDrive/Documents/python_ws/project_delta/data/processed_data/customer_information_data.csv")
df = pd.read_csv("/Users/Jamie/OneDrive/Documents/python_ws/project_delta/data/processed_data/sales_data.csv")

# Training Models (Regression)

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [83]:
# Step 3: Sort by transaction_date
df = df.sort_values(by=['transaction_date'])
df.set_index('transaction_date',inplace=True)

In [84]:
customer_info_df = customer_info_df.drop(columns=['Unnamed: 0', 'customer_id', 'churned', 'loyalty_program', 'membership_years'])

In [85]:
customer_info_df.head()

,age,gender,income_bracket,marital_status,number_of_children,education_level,occupation
0,56,Other,High,Divorced,3,Bachelor's,Self-Employed
1,69,Female,Medium,Married,2,PhD,Unemployed
2,46,Female,Low,Married,3,Bachelor's,Self-Employed
3,32,Female,Low,Divorced,2,Master's,Employed
4,60,Female,Low,Divorced,2,Bachelor's,Employed


In [88]:
def Label_encoder(df, columns):
    label_encoder = LabelEncoder()
    for column in columns:
        if column in df.columns and df[column].dtype == 'object':
            df[column] = label_encoder.fit_transform(df[column])
    return df

def OneHot_encoder(df, columns):
    df = pd.get_dummies(df, columns=columns)
    return df

# Usage example:
columns_to_label_encode = ['gender', 'marital_status', 'occupation']
columns_to_onehot_encode = ['income_bracket', 'education_level']
customer_info_df = Label_encoder(customer_info_df, columns_to_label_encode)
customer_info_df = OneHot_encoder(customer_info_df, columns_to_onehot_encode)

In [89]:
customer_info_df.head()

,age,gender,marital_status,number_of_children,occupation,income_bracket_High,income_bracket_Low,income_bracket_Medium,education_level_Bachelor's,education_level_High School,education_level_Master's,education_level_PhD
0,56,2,0,3,2,True,False,False,True,False,False,False
1,69,0,1,2,3,False,False,True,False,False,False,True
2,46,0,1,3,2,False,True,False,True,False,False,False
3,32,0,0,2,0,False,True,False,False,False,True,False
4,60,0,0,2,0,False,True,False,True,False,False,False


In [93]:
# spl

ValueError: not enough values to unpack (expected 4, got 2)